<a href="https://colab.research.google.com/github/jwang44/Try-colabing-in-colab/blob/main/ECSE01_2_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import numpy as np
import pandas as pd
import scipy.stats
import statistics

## The model

In [ ]:
def sigmoid(a):
  return 1/(1+np.exp(-a))

In [ ]:
class Logistic_regression():
  def __init__(self,X_train,y_train,learning_rate,X_test,y_test):
    self.X_train = X_train
    self.y_train = y_train
    self.learning_rate = learning_rate
    self.X_test = X_test
    self.y_test = y_test

  def fit(self):
    n,m = np.shape(self.X_train)
    itrnum = 500
    W = np.ones([m+1,itrnum+1])
    e = 0.01        
    der = 0
    for k in range(0,itrnum):
      for i in range(0,n):
        xi = self.X_train[i].T
        x0 = np.array([1])
        xi = np.concatenate((xi, x0),axis = 0)
        yi = self.y_train[i]
        der = der-xi*(yi-sigmoid(np.matmul(W[:,k].T,xi)))
      W[:,k+1] = W[:,k]-self.learning_rate*der
      if (np.linalg.norm(W[:,k+1]-W[:,k]))**2<e:
        break 
    return W[:,k+1]
  
  def predict(self):
    w = self.fit()
    n,m = np.shape(self.X_test)   
    y_predict = np.zeros([n,1])
    for i in range(0,n):
      xi = self.X_test[i].T
      x0 = np.array([1])
      xi = np.concatenate((xi, x0),axis = 0)
      p1 = sigmoid(np.matmul(w.T,xi))
      #p0=1-sigmoid(np.matmul(w.T,xi))
      if p1>=0.5:
        y_predict[i] = 1
      else:
        y_predict[i] = 0
    return y_predict

  '''def Accu_eval(self):
    y_predict = self.predict()
    n,i = np.shape(y_predict)
    err = np.sum(np.abs(y_predict-self.y_test))
    accuracy = 1-err/n
    return accuracy'''
  
  def Accu_eval(self):
    y_predict = self.predict()
    n,j = np.shape(y_predict)
    TP = 0
    FP = 0
    TN = 0
    FN = 0
    for i in range(n):
      if  self.y_test[i]==1 and y_predict[i]==1:
        TP = TP+1
      elif self.y_test[i]==0 and y_predict[i]==0:
        TN = TN+1
      elif self.y_test[i]==0 and y_predict[i]==1:
        FP = FP+1
      elif self.y_test[i]==1 and y_predict[i]==0:
        FN = FN+1
    accuracy = (TP+TN)/(TP+TN+FP+FN)
    precision = TP/(TP+FP)
    recall = TP/(TP+FN)
    F = 2*precision*recall/(precision+recall)
    specificity = TN/(FP+TN)
    FPR = FP/(FP+TN)
    print("accuracy:",accuracy)
    print("precision:",precision)
    print("recall:",recall)
    print("F:",F)
    print("specificity:",specificity)
    print("False Positive Rate:",FPR)
    print("")
    return accuracy
    

## Cross validation

In [ ]:
class Cross_validation():
  def __init__(self, k):
    # k: k-fold
    self.k = k

  def prepare_data(self, data):
    # data: np array converted from csv
    np.random.shuffle(data)
    X = data[:, :-1]  # features
    y = data[:, -1]   # labels

    # split data into k equal segments, assign them to train and test later
    Xs = np.array_split(X, self.k, axis=0)
    ys = np.array_split(y, self.k, axis=0)
    return Xs, ys

  def get_accuracy(self, Xs, ys, lr):
    accuracies = []
    for i in range(self.k):
      X_cv = Xs[:] # X_cross_validation
      y_cv = ys[:] # y_cross_validation

      X_test = X_cv.pop(i)
      y_test = y_cv.pop(i)

      X_train = np.concatenate(X_cv)
      y_train = np.concatenate(y_cv)

      logistic_regression = Logistic_regression(X_train, y_train, lr, X_test, y_test)

      print("----------FOLD ", i+1, "----------")
      accuracy = logistic_regression.Accu_eval()
      accuracies.append(accuracy)
    return np.mean(accuracies)
      # this will print the evaluation results

## Change directory

In [5]:
%cd "/content/drive/My Drive/"

/content/drive/My Drive


## Dataset

In [24]:
# generate new feature by multiplication and normalize
def newfeature(x,y):
  z=x*y
  norz=scipy.stats.zscore(z, axis=0, ddof=0, nan_policy='propagate')
  return norz

### Orthopedic Patients Dataset

In [25]:
# convert csv to dataframe
df = pd.read_csv('orthopedic_patients.csv')
# data = df.to_numpy()

In [26]:
# normalize feature
NorData = scipy.stats.zscore(df.iloc[:,:-1], axis=0, ddof=0, nan_policy='propagate')
# normalized dataset
NorDataset = np.column_stack((NorData, df['Class']))
# np.savetxt('normalized_orthopedic_patients.csv', NorPatientData, delimiter=',')

New feature

In [27]:
# new feature
NewF = newfeature(df.pelvic_incidence, df.sacral_slope)
# normalized dataset with new feature
NorDatasetNew = np.column_stack((NorData,NewF,df['Class']))

### Credit Card Dataset

In [28]:
# convert csv to dataframe
df = pd.read_csv('creditcard.csv')
# data = df.to_numpy()

In [29]:
# normalize feature
NorData = scipy.stats.zscore(df.iloc[:,:-1], axis=0, ddof=0, nan_policy='propagate')
# normalized dataset
NorDataset = np.column_stack((NorData,df.iloc[:,-1]))

New feature

In [30]:
NewF1 = newfeature(df.V3, df.V7)
NewF2 = newfeature(df.V11, df.V12)
NewF3 = newfeature(df.V12, df.V16)
NewF4 = newfeature(df.V16, df.V17)
NewF5 = newfeature(df.V16, df.V18)
NewF6 = newfeature(df.V17, df.V18)
# new feature
NewF = np.column_stack((NewF1,NewF2,NewF3,NewF4,NewF5,NewF6))

In [31]:
# normalized dataset with new feature
NorDatasetNew = np.column_stack((NorData,NewF,df.iloc[:,-1]))

## Experiment with different learning rates

In [ ]:
lrs = np.logspace(-6, -1, 6) # different learning rates to try
# or we can also try other hyperparameters here
cv = Cross_validation(5) # 5-fold cross-validation
Xs, ys = cv.prepare_data(data)
for lr in lrs:
  print("----------LEARNING RATE: ", lr, "----------")
  accu_avg = cv.get_accuracy(Xs, ys, lr)
  print("----------AVERAGE ACCURACY", accu_avg, "----------")
  print("\n---------------------------------------------------------------------\n")


----------LEARNING RATE:  1e-06  ----------
----------FOLD  1 ----------
accuracy: 0.25125628140703515
precision: 0.33783783783783783
recall: 0.49504950495049505
F: 0.40160642570281124
specificity: 0.0
False Positive Rate: 1.0

----------FOLD  2 ----------
accuracy: 0.30303030303030304
precision: 0.3918918918918919
recall: 0.5471698113207547
F: 0.4566929133858268
specificity: 0.021739130434782608
False Positive Rate: 0.9782608695652174

----------FOLD  3 ----------
accuracy: 0.2777777777777778
precision: 0.34591194968553457
recall: 0.5851063829787234
F: 0.4347826086956521
specificity: 0.0
False Positive Rate: 1.0

----------FOLD  4 ----------
accuracy: 0.21717171717171718
precision: 0.30656934306569344
recall: 0.4117647058823529
F: 0.35146443514644354
specificity: 0.010416666666666666
False Positive Rate: 0.9895833333333334

----------FOLD  5 ----------
accuracy: 0.24242424242424243
precision: 0.3032258064516129
recall: 0.5280898876404494
F: 0.38524590163934425
specificity: 0.009174311

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


accuracy: 0.9396984924623115
precision: 1.0
recall: 0.8811881188118812
F: 0.9368421052631579
specificity: 1.0
False Positive Rate: 0.0

----------FOLD  2 ----------
accuracy: 0.9292929292929293
precision: 0.9693877551020408
recall: 0.8962264150943396
F: 0.9313725490196079
specificity: 0.967391304347826
False Positive Rate: 0.03260869565217391

----------FOLD  3 ----------
accuracy: 0.9343434343434344
precision: 0.9764705882352941
recall: 0.8829787234042553
F: 0.9273743016759776
specificity: 0.9807692307692307
False Positive Rate: 0.019230769230769232

----------FOLD  4 ----------
accuracy: 0.9040404040404041
precision: 0.8487394957983193
recall: 0.9901960784313726
F: 0.9140271493212669
specificity: 0.8125
False Positive Rate: 0.1875

----------FOLD  5 ----------
accuracy: 0.9646464646464646
precision: 0.9880952380952381
recall: 0.9325842696629213
F: 0.9595375722543353
specificity: 0.9908256880733946
False Positive Rate: 0.009174311926605505

----------AVERAGE ACCURACY 0.934404344957108